## Notebook to mosaic images captured on the same day

Jukes Liu & Rainey Aberle

January 2022

In [ ]:
import os
import rasterio as rio
import numpy as np
import glob
import subprocess
from osgeo import gdal
from xml.dom import minidom
from rasterio.plot import show
import matplotlib.pyplot as plt

In [ ]:
### ---path to images--- ###
# impath = '/Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/PSScene4Band/'
impath = '/Users/raineyaberle/Research/PhD/Turner/Planet-imagery/order_71057ea6-fc00-4fb2-ae18-d2e18dac91d7'

### ---path for output images--- ###
# outpath = '/Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/'
outpath = impath

### ---extension for mosaicing--- ###
# ext = 'SR_clip'
ext = 'SR_harmonized_clip'

### Mosaic reflectance image files

In [ ]:
uniquescenes = [] # grab unique scenes from a folder with all scenefiles
for scene in os.listdir(impath):
    date = scene[0:8]
    uniquescenes.append(date)
uniquescenes = list(set(uniquescenes))
uniquescenes.sort() # sort chronologically
uniquescenes

In [ ]:
# make directory for SR stitched outputs
stitch_path = outpath+'SR-stitched/'
if os.path.isdir(stitch_path)==0:
    os.mkdir(stitch_path)
    print(stitch_path+' directory made')

# loop through unique scenes
for scene in uniquescenes:
    filepaths = [] # files from the same date to mosaic together
    for file in os.listdir(impath): # check all files
        if (scene in file) & (ext in file): # if they match the scene date and have the right extension
            filepaths.append(impath+file) # add the path to the file
    
    # construct the gdal_merge command
    cmd = 'gdal_merge.py -v '

    # add in input files
    for filepath in filepaths:
        cmd += filepath+' '
    
    # define the out path with correct extension
    if ext == 'DN_udm':
        out = os.path.join(stitch_path, scene + "_DN_mask.tif")
    elif ext == 'udm2':
        out = os.path.join(stitch_path, scene + "_mask.tif")
    else:
        out = os.path.join(stitch_path, scene + ".tif")

    cmd += '-o '+out

    # Run the command 
    p = subprocess.run(cmd, shell=True, capture_output=True) 
    print(p)

In [ ]:
# Print files in outpath to verify
os.listdir(stitch_path)

In [ ]:
# Plot image mosaics to preview
os.chdir(stitch_path)
im_names = glob.glob('*.tif')
im_names.sort()
for im_name in im_names:
    im = rio.open(im_name)
    n = im.read(4)
    plt.title(im_name[0:8])
    plt.imshow(n)
    plt.clim(0,15000)
    plt.colorbar()
    plt.show()

### Done!